# Project
- 70% des utilisateurs souhaitent plus d'informations à propos de leur destination
- Chercher les meilleures villes en fonction de la météo 
- Proposer des hôtels sur ces villes

In [2]:
import requests
import json
import pandas as pd  

In [3]:
url = 'https://data.geopf.fr/geocodage/search'

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3 ',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

payload = {
    'q': 'Mont Saint Michel', 'France'
    'city': 'Mont Saint Michel',
    'format': 'json'
}

response = requests.get(url=url, params=payload, headers=headers)
response.json()


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [5.995538, 45.546586]},
   'properties': {'label': 'Mont Saint Michel 73190 Challes-les-Eaux',
    'score': 0.9464936363636363,
    'type': 'locality',
    'importance': 0.41143,
    'id': '73064_B039',
    'banId': 'f67c5810-39e4-4a3d-a92c-89380b8e6ab3',
    'name': 'Mont Saint Michel',
    'postcode': '73190',
    'citycode': '73064',
    'x': 933687.63,
    'y': 6498554.9,
    'city': 'Challes-les-Eaux',
    'context': '73, Savoie, Auvergne-Rhône-Alpes',
    'locality': 'Mont Saint Michel',
    '_type': 'address'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.538198, 47.243963]},
   'properties': {'label': 'Avenue du Mont Saint Michel 44300 Nantes',
    'score': 0.6999336363636363,
    'id': '44109_5785',
    'banId': 'bfc8760b-6503-436a-b194-96398ee4dd18',
    'name': 'Avenue du Mont Saint Michel',
    'postcode': '44300',
    'citycode': '44

In [5]:
# Liste des villes à interroger
villes = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]



base_url = "https://data.geopf.fr/geocodage/search?"


def get_coordinates(ville):
    
    params = {
        'q': ville,
        'limit': 1,
        'index':'poi'
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête pour {ville}: {response.status_code}")
        return None


results = []
for ville in villes:
    coordinates = get_coordinates(ville)
    if coordinates:
        results.append({'Ville': ville, 'Longitude': coordinates[0], 'Latitude': coordinates[1]})
    else:
        results.append({'Ville': ville, 'Longitude': None, 'Latitude': None})


In [6]:
response.json()['features'][0]['geometry']['coordinates'] #recherche du chemin pour extraire les coordonnées GPS

[5.995538, 45.546586]

In [7]:
df = pd.DataFrame(results) # transformation des données en dataframe
df.head()

,Ville,Longitude,Latitude
0,Mont Saint Michel,7.088900,43.849841
1,St Malo,-2.000385,48.653039
2,Bayeux,-0.702368,49.277173
3,Le Havre,0.128407,49.493515
4,Rouen,1.099092,49.443440


In [8]:
df.shape # verification des dimensions savoir si on a bien 35 villes 

(35, 3)

In [9]:
df.isnull().any() #verification des valeurs nulles

Ville        False
Longitude    False
Latitude     False
dtype: bool

In [10]:
df.loc[31]

Ville        Montauban
Longitude     1.366155
Latitude     44.022919
Name: 31, dtype: object

In [11]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", text="Ville",
                        zoom=4, 
                        mapbox_style="open-street-map",
                        title="Verification si mes coordonnées reçues sont bien en France")
fig.update_traces(marker=dict(size=12))
fig.show()

### Get weather infos

In [12]:
base_weather_url = 'https://api.openweathermap.org/data/2.5/onecall?'

def get_weather(lat, lon):
    params = {
        'lat': lat,
        'lon': lon,
        'appid': '3f80d368ad5b5238c109fb9243411a4d',
        'units': 'metric',
        'exclude':['current','minutely','hourly','alerts']  
    }
    
    response = requests.get(base_weather_url, params=params)
    return response.json() if response.status_code == 200 else None

print(response.json())



{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [5.995538, 45.546586]}, 'properties': {'label': 'Mont Saint Michel 73190 Challes-les-Eaux', 'score': 0.9464936363636363, 'type': 'locality', 'importance': 0.41143, 'id': '73064_B039', 'banId': 'f67c5810-39e4-4a3d-a92c-89380b8e6ab3', 'name': 'Mont Saint Michel', 'postcode': '73190', 'citycode': '73064', 'x': 933687.63, 'y': 6498554.9, 'city': 'Challes-les-Eaux', 'context': '73, Savoie, Auvergne-Rhône-Alpes', 'locality': 'Mont Saint Michel', '_type': 'address'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-1.538198, 47.243963]}, 'properties': {'label': 'Avenue du Mont Saint Michel 44300 Nantes', 'score': 0.6999336363636363, 'id': '44109_5785', 'banId': 'bfc8760b-6503-436a-b194-96398ee4dd18', 'name': 'Avenue du Mont Saint Michel', 'postcode': '44300', 'citycode': '44109', 'x': 356904.59, 'y': 6692490.84, 'city': 'Nantes', 'context': '44, Loire-Atlantique, Pays

In [14]:
# Test with the first city
test_city = df.iloc[0]
weather_data = get_weather(test_city['Latitude'], test_city['Longitude'])
print(weather_data)

None
